In [1]:

%pip install ipywidgets



Note: you may need to restart the kernel to use updated packages.


Hämta namn till städer, berg och realm med sql från existerande databas

In [1]:
# Query to select all data from the names table
import sqlite3, csv


conn = sqlite3.connect('database.db') # Connecting to a database (and creating it if it doesn't exist)
cursor = conn.cursor() # Creating a cursor object
cursor.execute('SELECT city, mountain, realm FROM names')

# Fetch all rows from the query
rows = cursor.fetchall()

cities = [row[0] for row in rows]
mountains = [row[1] for row in rows]
realms = [row[2] for row in rows]


In [2]:
# Generate continents
import random
from Classes.map import Map
from IPython.display import display, clear_output
import draw_PNG
import ipywidgets as widgets
import time
import pickle



# Create sliders
slider1 = widgets.IntSlider(
    min=1,     # minimum value
    max=50,    # maximum value
    value=10,   # default value
    step=1,    # step size
    description='Continents',
    continuous_update=True
)

slider2 = widgets.IntSlider(
    min=10,     # minimum value
    max=100,    # maximum value
    value=50,   # default value
    step=1,    # step size
    description='Continent scale',
    continuous_update=True
)

slider3 = widgets.IntSlider(
    min=0,     # minimum value
    max=100,    # maximum value
    value=50,   # default value
    step=1,    # step size
    description='River scale',
    continuous_update=True
)

slider4 = widgets.IntSlider(
    min=1,     # minimum value
    max=100,    # maximum value
    value=50,   # default value
    step=1,    # step size
    description='Mountain scale',
    continuous_update=True
)

slider5 = widgets.IntSlider(
    min=1,     # minimum value
    max=100,    # maximum value
    value=50,   # default value
    step=1,    # step size
    description='Village scale',
    continuous_update=True
)


# Create a button widget
button1 = widgets.Button(
    description='Generate Image',
    button_style='success' 
)

button2 = widgets.Button(
    description='Save image',
    button_style='danger'  
)

text_input = widgets.Text(
    description='File name',
    value='', 
    placeholder='Map'
)

output = widgets.Output()
done = False

karta = ''

def on_button_click1(b):
    global karta
    with output:
        done = False
        clear_output(wait=True)
        amount_continents = slider1.value
        continent_scale = slider2.value
        river_scale = slider3.value
        mountain_scale = slider4.value
        village_scale = slider5.value
        print('Generating picture, please wait')
        
        realm_name = random.choice(realms)
        karta = Map(amount_continents, continent_scale, river_scale, mountain_scale, village_scale, mapsize = (1000, 1000), realm_name = realm_name, village_names = cities, mountain_names = mountains)
        draw_PNG.draw_map(karta)
        clear_output(wait=True)
        done = True
        image = open("test_image.png", "rb").read() 
    
        image_widget = widgets.Image(
            value=image,
            format='png',
            width=1000,
            height=1000
        )
        display(image_widget)

def on_button_click2(b):
    global karta
    with output:
        entered_text = text_input.value  
        path = 'Maps/' + entered_text + '.pkl'
        with open(path, 'wb') as file:
            pickle.dump(karta, file)
            print(f"Saved succesfully")

        



button1.on_click(on_button_click1)

button2.on_click(on_button_click2)

hbox1 = widgets.HBox([slider1, slider2, slider3])
hbox2 = widgets.HBox([slider4, slider5])

display(hbox1, hbox2, button1, output, button2, text_input)


Button(button_style='success', description='Generate Image', style=ButtonStyle())

Output()

Button(button_style='danger', description='Save image', style=ButtonStyle())

Text(value='', description='File name', placeholder='Map')

Fetch old maps from the database


In [12]:
button3 = widgets.Button(
    description='Generate loaded map',
    button_style='success' 
)

file_upload = widgets.FileUpload(
    accept='.pkl',  
    multiple=False 
)

karta_uploaded_pickle = None

# Function to handle the file upload
def on_file_upload(change):
    global karta_uploaded_pickle
    with output1:
        data = file_upload.value[0]
        karta_uploaded_pickle = data['name']



def on_button_click3(b):
    global karta_uploaded_pickle
    with output1:
        if karta_uploaded_pickle:
            clear_output(wait=True)
            karta_loaded_path = 'Maps/' + karta_uploaded_pickle
            with open(karta_loaded_path, 'rb') as file:
                karta = pickle.load(file)
            draw_PNG.draw_map(karta)
            image = open("test_image.png", "rb").read() 
    
            image_widget = widgets.Image(
                value=image,
                format='png',
                width=1000,
                height=1000
            )
            display(image_widget)
             
        else:
            print('Choose a map first')
        

file_upload.observe(on_file_upload, names='value')
button3.on_click(on_button_click3)
output1 = widgets.Output()
display(file_upload, button3, output1)

FileUpload(value=(), accept='.pkl', description='Upload')

Button(button_style='success', description='Generate loaded map', style=ButtonStyle())

Output()

Connecting to database and reading city, mountain and realm names from a CSV

In [2]:
import sqlite3, csv

conn = sqlite3.connect('database.db') # Connecting to a database (and creating it if it doesn't exist)
cursor = conn.cursor() # Creating a cursor object
cursor.execute("DROP TABLE IF EXISTS names") # Deleting table so that it can be created anew
cursor.execute("CREATE TABLE names (city TEXT, mountain TEXT, realm TEXT)")

# Read the CSV file and insert data into the table
with open('fictitiousnames.csv', mode='r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        cursor.execute('''
        INSERT INTO names (city, mountain, realm) VALUES (?, ?, ?)
        ''', (row['city'], row['mountain'], row['realm']))

# Makes sure the entire transaction is comitted.
# This means that if there is a problem in any of the queries above, none of them will be executed.
# This, in turn, helps prevent partial updates that can lead to data inconsistency
conn.commit()

If we want to we can also enter some extra data into out database

In [3]:
cursor.execute('INSERT INTO names (city, mountain, realm) VALUES (?, ?, ?)', ("Aastertonia", "Eversnow", "Utopia"))
conn.commit()

Let's have a look at the output

In [5]:
# Querying the data
cursor.execute('SELECT * FROM names ORDER by city LIMIT 10')

# Fetching and displaying the results
rows = cursor.fetchall()
for row in rows:
    print(row)

('Aastertonia', 'Eversnow', 'Utopia')
('Aethervale', 'Ashencliff', 'Vaeloria')
('Aetherwatch', 'Firewatch Peak', 'Elandor')
('Amberwood', 'Windshear Peak', 'Valindorithis')
('Ashbrook', 'Flamecliff', 'Azaroth')
('Ashenford', 'Stormclaw Ridge', 'Arendor')
('Ashenhold', 'Ironfall Mountain', 'Eldrasis')
('Ashenpeak', 'Frostwatch Ridge', 'Zynthorion')
('Ashenvale', 'Emberreach', 'Braxis')
('Ashglen', 'Frostreach', 'Valandris')


In [6]:
# Query to select all data from the names table
cursor.execute('SELECT city, mountain, realm FROM names')

# Fetch all rows from the query
rows = cursor.fetchall()

cities = [row[0] for row in rows]
mountains = [row[1] for row in rows]
realms = [row[2] for row in rows]

print("Cities:", cities)
print("Mountains:", mountains)
print("Realms:", realms)

Cities: ['Thornpeak', 'Crystalhold', 'Emberstone', 'Mistwatch', 'Thorncrest', 'Goldencrest', 'Luminous Heights', 'Ironpeak', 'Sunspire', 'Willowmist', 'Thistlethorn', 'Mistvale', 'Raventhorn', 'Pinecrest', 'Dreadhaven', "Prowler's Peak", 'Starwood', 'Firepeak', 'Jewelspire', 'Wraithwood', 'Thornwatch', 'Grimstone', 'Willowdale', 'Juniper Hollow', 'Valewood', 'Emberveil', 'Windwatch', 'Hawkthorn', 'Fireglen', 'Starfall', 'Firevale', 'Frostspire', 'Silvercrest', 'Umbercliff', 'Valemyst', 'Frostpeak', 'Frostvale', 'Grimwood', "Kestrel's Rest", 'Moonshadow', 'Nightglade', 'Dragonspire', 'Thistlewood', 'Zenithal Heights', 'Windspire', 'Everwinter', 'Pinefall', 'Winterhold', 'Nightglen', 'Brightspire', 'Thornreach', 'Netherhaven', 'Darkvale', 'Stormspire', 'Verdant Hollow', 'Windpeak', 'Crystalglen', 'Grimshade', 'Starstone', 'Ravenwatch', 'Duskwood', 'Glacial Keep', 'Ashhaven', 'Moonbrook', 'Windstone', 'Brightreach', 'Fernhollow', 'Fireglow', 'Frostveil', 'Hollowforge', 'Cragspire', 'Quiet

In [ ]:
# Closing the connection when done. Good for resource managment
conn.close()

In [41]:
text2 = text2.replace("\n",", ")

In [42]:
text = "Aeloria, Brighthaven, Cragspire, Elderglen, Frosthaven, Gloomvale, Hightower, Ironwatch, Jadespire, Kaelthar, Larkwood, Mistyvale, Nighthaven, Oakhurst, Pinefall, Quicksilver, Ravenstone, Shadowfen, Thundertop, Vespera, Windwhisper, Yewwood, Zephyrhold, Ashvale, Brightwater, Cinderfall, Dragonreach, Emberglow, Fallowdeep, Graywatch, Hollowbend, Ivorymere, Jewelspire, Kestrel's Rest, Luminous Bay, Mooncrest, Netherwood, Oakenshield, Prowler's Peak, Quietgrove, Rivenwood, Starfall, Thornfield, Umbercliff, Verdant Hollow, Willowmist, Aethervale, Blackridge, Crestfall, Deepwood, Eldenmoor, Fernhollow, Glimmerstone, Hearthglen, Ironroot, Jadedale, Kingsford, Lonesome Hollow, Mistral Heights, Nightshade, Oakshadow, Pinewatch, Quagmire, Raven's Nest, Silvershadow, Tanglewood, Upland Falls, Viper's Hollow, Wraithwood, Yonderhill, Azurecrown, Blackthorn, Cloudspire, Duskwood, Emberforge, Frostglade, Goldenshade, Harrowgate, Ironpeak, Juniper Hollow, Keepstone, Lumina, Mistveil, Netherbrook, Oakhaven, Pinecrest, Quicksand, Redstone, Silverwind, Thistlewood, Underbough, Vesperwood, Willowdale, Aetherwatch, Brimstone, Cinderbrook, Darkwater, Eldridge, Fernspire, Glimmerveil, Hawthorn Grove, Ivoryspire, Jadehollow, Kingswood, Larkspire, Moonshadow, Netherhaven, Osprey's Perch, Pearlwood, Quillhaven, Ravenscar, Silverbloom, Thornspire, Umberveil, Valewood, Wyrmrest, Zenithal Heights, Amberwood, Briarthorn, Cobalt Cove, Dawnspire, Eldenwood, Frostglen, Glacial Keep, Hollowshade, Ironspring, Juniper Heights, Kingfisher, Luminous Vale, Mistwood, Nightfall, Oakridge, Prowler's Roost, Quietwood, Redcliff, Stonehaven, Thundermark, Umberhaven, Verdant Reach, Willowbark, Ashenhold, Blackwood, Crystal Bay, Driftwood, Emberstone, Fernvale, Goldencrest, Harrowmoor, Ironhill, Juniper Bend, Kindling, Larkfall, Moonstone, Netherfield, Oakenbriar, Pinevale, Quicksilver Cove, Ravenwood, Silvershadow, Thistlethorn, Uplands, Vespergate, Wraithfell, Yewshade, Azurewood, Briarstone, Cinderscar, Dawnwood, Elderspring, Frostwold, Glimmerstone, Hightide, Ironscar, Juniper Vale, Kingsreach, Luminous Heights, Mistvale, Nightbloom, Oakenvale, Pinehill, Quagmire Bay, Ravenstone, Silverhollow, Thornfield, Underveil, Viper's Den, Wyrmwood, Zenith Vale"

In [43]:
text = text + text2

In [44]:
lst = text.split(", ")
my_set= set(x for x in lst if x != "")
print(len(lst))
print(len(my_set))

417
391


In [45]:
print("\n".join(my_set))

Thornpeak
Crystalhold
Emberstone
Mistwatch
Thorncrest
Goldencrest
Luminous Heights
Ironpeak
Sunspire
Willowmist
Thistlethorn
Mistvale
Raventhorn
Pinecrest
Dreadhaven
Prowler's Peak
Starwood
Firepeak
Jewelspire
Wraithwood
Thornwatch
Grimstone
Willowdale
Juniper Hollow
Valewood
Emberveil
Windwatch
Hawkthorn
Fireglen
Starfall
Firevale
Frostspire
Silvercrest
Umbercliff
Valemyst
Frostpeak
Frostvale
Grimwood
Kestrel's Rest
Moonshadow
Nightglade
Dragonspire
Thistlewood
Zenithal Heights
Windspire
Everwinter
Pinefall
Winterhold
Nightglen
Brightspire
Thornreach
Netherhaven
Darkvale
Stormspire
Verdant Hollow
Windpeak
Crystalglen
Grimshade
Starstone
Ravenwatch
Duskwood
Glacial Keep
Ashhaven
Moonbrook
Windstone
Brightreach
Fernhollow
Fireglow
Frostveil
Hollowforge
Cragspire
Quietgrove
Jadehollow
Sylvalis
Goldenleaf
Embercrest
Crystalbrook
Zephyrhold
Ivoryspire
Embervale
Shadowcrest
Ashenford
Grimspire
Cobalt Cove
Quicksilver
Sunwatch
Brightwater
Brighthaven
Emberforge
Nightspire
Mistveil
Larkspire
